In [7]:
import os
import json

def preprocess_code_line(line: str) -> str:
    """
    对源码的单行做轻度预处理：
    1. rstrip() 去掉行尾空格
    2. 若含 '#', 去掉 '#' 后面的内容（行尾注释）
    3. strip() 再次去除前后空白
    """
    line = line.rstrip()
    if '#' in line:
        line = line.split('#', 1)[0]
    line = line.strip()
    return line

def find_substring_in_original_lines(
    original_lines: list[str],
    block_label: str,
    start_offset: int = 0
) -> tuple[int, int]:
    """
    在 original_lines 中查找 block_label 对应的文本段位置，
    返回 (start_line, end_line) —— 对应**原始文件的真实行号**。

    具体策略：
      1) 为每行生成 (原行索引, 预处理后文本) 的列表 preproc_index，
         不移除原行，只是“在匹配时”忽略那些预处理后变成空的行。
      2) 同样地，对 block_label 做多行拆分并预处理为 preproc_label。
      3) 在 preproc_index 上做滑动窗口匹配，如果某段连续文本与 preproc_label 一致，
         则返回那段的 (原行索引+1) 范围。
      4) 如果匹配不到，返回 (None, None).

    参数:
      original_lines: 原文件的所有行(含换行拆分后)，下标从 0 开始
      block_label:    LLM CFG 中的原始 label(可能多行)
      start_offset:   表示 original_lines[0] 在原文件中的行号-1; 常用于函数/类的行号范围。
                      如果你要全文件搜索，就令 start_offset=0。

    注意:
      - 这里不会修改 block_label，只是临时预处理一份来匹配
      - 找到的行号是原文件的真实行号( start_offset + (匹配到的index) + 1 )，而不是去掉空行后的行数
    """

    # 1) 生成 (orig_idx, processed_text) 的列表，过滤掉处理后为空的行
    #    这样可以“忽略”空行或仅有注释的行，在匹配中跳过。
    preproc_index = []  # [(原文件下标, 预处理后的字符串), ...]
    for i, line in enumerate(original_lines):
        processed = preprocess_code_line(line)
        if processed:  # 非空才参与匹配
            preproc_index.append((i, processed))

    # 2) 预处理 block_label
    block_label_lines = block_label.split('\n')
    preproc_label = []
    for lbl_line in block_label_lines:
        pl = preprocess_code_line(lbl_line)
        if pl:
            preproc_label.append(pl)

    if not preproc_label:
        # 如果预处理后 label 为空，那没法匹配
        return (None, None)

    # 3) 在 preproc_index 上做滑动窗口，寻找连续匹配
    needed = len(preproc_label)
    for start_idx in range(len(preproc_index) - needed + 1):
        match_all = True
        for offset in range(needed):
            if preproc_index[start_idx + offset][1] != preproc_label[offset]:
                match_all = False
                break
        if match_all:
            # 匹配成功
            # preproc_index[start_idx].0 是匹配开始行在原文件的下标
            # preproc_index[start_idx + needed - 1].0 是匹配结束行在原文件的下标
            orig_start = preproc_index[start_idx][0]
            orig_end   = preproc_index[start_idx + needed - 1][0]
            # +1，因为原文件行号从1开始
            start_line = start_offset + orig_start + 1
            end_line   = start_offset + orig_end   + 1
            return (start_line, end_line)

    # 没找到
    return (None, None)

def annotate_block_and_successors(block: dict, lines_in_range: list[str], offset: int):
    """
    给当前 block 及其 successors 标注 start_line/end_line。
    - 仅在匹配时做预处理
    - 行号使用原文件(含空行、注释)的真实下标
    参数:
      block        : 当前 block
      lines_in_range: 这一层(函数/类/全文件)内可搜索的源码行
      offset       : lines_in_range[0] 在原文件中的行号-1
                     如果是全文件，就 offset=0；
                     如果是函数/类，则 offset=函数/类起始行号-1
    """
    block_label = block.get("label", "")
    s_line, e_line = find_substring_in_original_lines(lines_in_range, block_label, start_offset=offset)

    block["start_line"] = s_line
    block["end_line"]   = e_line

    # 递归处理 successors
    for succ in block.get("successors", []):
        annotate_block_and_successors(succ, lines_in_range, offset)


def annotate_llm_cfg_with_lineinfo_recursive(llm_cfg: dict, static_cfg: dict, source_lines: list[str]):
    """
    分层对齐：
      - 从 static_cfg 里拿到函数/类的 (start_line, end_line)
      - 只在那段行范围里搜索 LLM CFG 的 blocks
      - 行号最终是原文件真实行号，不受空行/注释干扰
    """

    # 顶层 blocks: 全文件范围
    full_start, full_end = 1, len(source_lines)
    top_level_segment = source_lines[full_start - 1 : full_end]

    for block in llm_cfg.get("blocks", []):
        annotate_block_and_successors(block, top_level_segment, full_start - 1)

    # 处理函数
    static_funcs = {}
    for f_obj in static_cfg.get("functions", []):
        fname = f_obj.get("name", "")
        if not fname:
            continue
        f_start = f_obj.get("start_line", 1)
        f_end   = f_obj.get("end_line", len(source_lines))
        static_funcs[fname] = (f_start, f_end, f_obj)

    for f_obj_llm in llm_cfg.get("functions", []):
        fname = f_obj_llm.get("name", "")
        if fname in static_funcs:
            (f_start, f_end, f_obj_static) = static_funcs[fname]
            func_segment = source_lines[f_start - 1 : f_end]
            # 标注 blocks
            for block in f_obj_llm.get("blocks", []):
                annotate_block_and_successors(block, func_segment, f_start - 1)
            # 递归
            annotate_llm_cfg_with_lineinfo_recursive(f_obj_llm, f_obj_static, source_lines)

    # 处理类
    static_classes = {}
    for c_obj in static_cfg.get("classes", []):
        cname = c_obj.get("name", "")
        if not cname:
            continue
        c_start = c_obj.get("start_line", 1)
        c_end   = c_obj.get("end_line", len(source_lines))
        static_classes[cname] = (c_start, c_end, c_obj)

    for c_obj_llm in llm_cfg.get("classes", []):
        cname = c_obj_llm.get("name", "")
        if cname in static_classes:
            (c_start, c_end, c_obj_static) = static_classes[cname]
            class_segment = source_lines[c_start - 1 : c_end]
            # 标注 blocks
            for block in c_obj_llm.get("blocks", []):
                annotate_block_and_successors(block, class_segment, c_start - 1)
            # 递归
            annotate_llm_cfg_with_lineinfo_recursive(c_obj_llm, c_obj_static, source_lines)

    return llm_cfg


def main():
    source_code_dir = "../../dataset/python"
    static_cfg_dir  = "../../dataset/python_cfg"
    llm_cfg_dir     = "merged_llm_cfg"
    add_line_cfg_dir= "add_line_cfg"

    os.makedirs(add_line_cfg_dir, exist_ok=True)

    # 根据文件名循环处理，比如 0.py ~ 199.py
    for i in range(200):
        py_path   = os.path.join(source_code_dir,  f"{i}.py")
        s_cfg_path= os.path.join(static_cfg_dir,   f"{i}.json")
        l_cfg_path= os.path.join(llm_cfg_dir,      f"{i}.json")

        if not (os.path.exists(py_path) and os.path.exists(s_cfg_path) and os.path.exists(l_cfg_path)):
            continue

        # 读源码
        with open(py_path, "r", encoding="utf-8") as f:
            source_lines = f.read().splitlines()

        # 读静态 CFG
        with open(s_cfg_path, "r", encoding="utf-8") as f:
            static_cfg = json.load(f)

        # 读 LLM CFG
        with open(l_cfg_path, "r", encoding="utf-8") as f:
            llm_cfg = json.load(f)

        # 分层标注行号（预处理仅用于搜索，但最终行号基于 original_lines 的真实下标）
        annotate_llm_cfg_with_lineinfo_recursive(llm_cfg, static_cfg, source_lines)

        # 保存
        out_path = os.path.join(add_line_cfg_dir, f"{i}.json")
        with open(out_path, "w", encoding="utf-8") as fout:
            json.dump(llm_cfg, fout, indent=2, ensure_ascii=False)

        print(f"已处理并保存：{out_path}")


if __name__ == "__main__":
    main()


已处理并保存：add_line_cfg/0.json
已处理并保存：add_line_cfg/1.json
已处理并保存：add_line_cfg/2.json
已处理并保存：add_line_cfg/3.json
已处理并保存：add_line_cfg/4.json
已处理并保存：add_line_cfg/5.json
已处理并保存：add_line_cfg/6.json
已处理并保存：add_line_cfg/7.json
已处理并保存：add_line_cfg/8.json
已处理并保存：add_line_cfg/9.json
已处理并保存：add_line_cfg/10.json
已处理并保存：add_line_cfg/11.json
已处理并保存：add_line_cfg/12.json
已处理并保存：add_line_cfg/13.json
已处理并保存：add_line_cfg/14.json
已处理并保存：add_line_cfg/15.json
已处理并保存：add_line_cfg/16.json
已处理并保存：add_line_cfg/17.json
已处理并保存：add_line_cfg/18.json
已处理并保存：add_line_cfg/19.json
已处理并保存：add_line_cfg/20.json
已处理并保存：add_line_cfg/21.json
已处理并保存：add_line_cfg/22.json
已处理并保存：add_line_cfg/23.json
已处理并保存：add_line_cfg/24.json
已处理并保存：add_line_cfg/25.json
已处理并保存：add_line_cfg/26.json
已处理并保存：add_line_cfg/27.json
已处理并保存：add_line_cfg/28.json
已处理并保存：add_line_cfg/29.json
已处理并保存：add_line_cfg/30.json
已处理并保存：add_line_cfg/31.json
已处理并保存：add_line_cfg/32.json
已处理并保存：add_line_cfg/33.json
已处理并保存：add_line_cfg/34.json
已处理并保存：add_line_cfg/35.json
已处